## API GraphQL de DynamoDB en AWS AppSync

AWS AppSync permite a los desarrolladores conectar sus aplicaciones y servicios a datos y eventos con API GraphQL y Pub/Sub seguras, sin servidor y de alto desempeño. Puede hacer lo siguiente con AWS AppSync:

* Acceder a datos de una o varias fuentes de datos desde un único punto de enlace de API GraphQL.

* Combinar varias API GraphQL de origen en una única API GraphQL fusionada.

* Publique actualizaciones de datos en tiempo real en sus aplicaciones.

* Aproveche la seguridad, la supervisión, el registro y el rastreo integrados, con almacenamiento en caché opcional para una baja latencia.

* Pague únicamente por las solicitudes de API y los mensajes en tiempo real que se entreguen.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime
import time

In [2]:
dt=DynamoTable()
try:
    dt.select_table('Books')
    print(dt)
except:
    dt.create_table(
        table_name='Books',
        partition_key='ISBN',
        partition_key_type='S'
    )

Table created successfully!


In [3]:
%%writefile bookdata.json
[
    {
        "ISBN": "978-1982121089",
        "Author": "John Grisham",
        "Title": "The Rainmaker",
        "Category": "Suspense"
    },
    {
        "ISBN": "978-1982121090",
        "Author": "William Shakespeare",
        "Title": "The Tempest",
        "Category": "Romance"
    },
    {
        "ISBN": "978-1982121091",
        "Author": "Charles Dickens",
        "Title": "Great Expectations",
        "Category": "Romance"
    },
    {
        "ISBN": "978-1982121092",
        "Author": "James Patterson",
        "Title": "Along Came a Spider",
        "Category": "Suspense"
    },
    {
        "ISBN": "978-1982121093",
        "Author": "Dr. Seuss",
        "Title": "Green Eggs and Ham",
        "Category": "Children"
    },
    {
        "ISBN": "978-1982121094",
        "Author": "Julie Simon",
        "Title": "Learn Amazon SageMaker",
        "Category": "Technology"
    }
]

Overwriting bookdata.json


In [4]:
dt.load_json('bookdata.json')

Data loaded successfully from bookdata.json.


### AppSync API

Vamos a crear una API seleccionando Build from Scratch, darle un nombre y crearla. AppSync similar a la mayoría de graphQL API, si usted está familiarizado con uno, tiene;

* **Fuentes de datos:** Fuente de los datos en AppSync api. Conectaremos dynamoDB mediante esta función. Hay varias fuentes de datos que podemos seleccionar como; Elastic Search, Lambda Function, Bases de datos relacionales, etc.

* **Esquema:** Schema es donde creamos qué tipos de datos hay y qué resolver para activar cuando se solicitan ciertas consultas, mutación de GraphQL API.

* **Funciones:** Las funciones son operaciones que podemos hacer contra una fuente de datos. Podemos adjuntar funciones como un resolver después de crear una operación dentro del esquema. Veremos esto más adelante en esta historia.

En primer lugar, creemos una fuente de datos y definámosla como la tabla `Books` creada anteriormente.

In [ ]:
schema {
    query: Query
    mutation: Mutation
}

type Query {
    getPost(ISBN: ID): Post
}

type Mutation {
    addPost(
        ISBN: ID!
        Author: String!
        Title: String!
        Category: String!
    ): Post!
}

type Post {
    ISBN: ID!
    Author: String
    Title: String
    Category: String
    ups: Int!
    downs: Int!
    version: Int!
}

#### Crear una función

Escribamos el código del resolvedor para obtener libros, que es una operación de Escaneo contra nuestra fuente de datos que creamos anteriormente. Pero como no podemos adjuntar una fuente de datos directamente a los resolvers de la línea de tuberías, necesitamos crear Funciones. Vayamos al menú de funciones de la barra lateral y creemos una función. Al crear una función, necesitamos utilizar una fuente de datos y un código de función. [Este documento](https://docs.aws.amazon.com/appsync/latest/devguide/js-resolver-reference-dynamodb.html#js-aws-appsync-resolver-reference-dynamodb-scan) contiene algunas de las operaciones que podemos realizar contra la fuente de datos dynamoDB y su formato de resultados.

Este es el código de la función:

In [ ]:
import { util } from '@aws-appsync/utils';

export function request(ctx) {
    return {
        operation: "Scan",
    };
}

export function response(ctx) {
    return ctx.result;
}